In [ ]:
!python whisperPrepWavs.py --WavFolder PathToYourFolder

# WARNING!!! This notebook is NOT user friendly or well documented.
# We don't have time, resources, and many experiments are still underway. 
# The times we live in, the extreme competition, the open source movement seems to be dying, 
# Really makes you wonder. So yeah... Whoever can read the code and design intent, this is christmas enough already.
# As Ellen from the Witch's House says... people are only kind when they can afford to be.

# Dataset prep

In [ ]:
import os
# Anyone else should probably change this to 0 if they only have one GPU. 
# We're only putting it on our secondary GPU since it has alot of VRAM. Training this thing takes ALOT of VRAM.
#Maybe we should give the authors the benefit of the doubt. Maybe it's not gatekeeping.
# Maybe it's not so much the intense desperate competition for an edge in this post-job world taken over by AI.
# Maybe the reason training code isn't being released is... 
# there is no way the average person is training this on their consumer GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"



import csv
import librosa
import torch
import numpy as np
from tqdm import tqdm
from transformers import SeamlessM4TFeatureExtractor, Wav2Vec2BertModel
from langid import classify
from utils.util import load_config
from maskgct.g2p.g2p_generation import g2p, chn_eng_g2p
from models.codec.kmeans.repcodec_model import RepCodec

from pytorch_optimizer import AdamW# Pick an optimizer. any optimizer. Well.. not ANY, but you get the idea

from transformers import WhisperProcessor, WhisperForConditionalGeneration
import LangLib

# Choose an appropriate Whisper model size based on your computational resources
#model_name = "openai/whisper-base"  # Options: tiny, base, small, medium, large
#whisper_processor = WhisperProcessor.from_pretrained(model_name)
#whisper_model = WhisperForConditionalGeneration.from_pretrained(model_name)
#whisper_model.eval()
#whisper_model.to(device)

#EDIT: Yeah, we're running the whisper stage separately. To conserve VRAM. 
# (In fact, don't bother using whisper during inferencing. Just type out the reference text yourself. )
# Otherwise you will not have enough VRAM

DatasetDir = "./Dataset/"
input_text_file = DatasetDir + "newbark-train.txt"

# Initialize necessary models and processors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load processor
processor = SeamlessM4TFeatureExtractor.from_pretrained("./w2v-bert-2.0")
# Load semantic model
semantic_model = Wav2Vec2BertModel.from_pretrained("./w2v-bert-2.0")
semantic_model.eval()
semantic_model.to(device)

# Load semantic mean and std
stat_mean_var = torch.load("./maskgct/ckpt/wav2vec2bert_stats.pt")
semantic_mean = stat_mean_var["mean"].to(device)
semantic_std = torch.sqrt(stat_mean_var["var"]).to(device)

# Load semantic codec
cfg_path = "./maskgct/config/maskgct.json"
cfg = load_config(cfg_path)
semantic_codec = RepCodec(cfg=cfg.model.semantic_codec)
semantic_codec.eval()
semantic_codec.to(device)
# Load semantic codec checkpoint
semantic_codec_ckpt = "./ckpt/MaskGCT/semantic_codec/model.safetensors"
import safetensors
safetensors.torch.load_model(semantic_codec, semantic_codec_ckpt)

# G2P functions
def detect_text_language(text):
    return classify(text)[0]

def g2p_(text, language):
    if language in ["zh", "en"]:
        return chn_eng_g2p(text)
    else:
        return g2p(text=text, sentence=None, language=language)


import json
 
        
def update_vocab(new_phonemes, vocab_file_path="./vocab.json"):
    # Load the existing vocab file
    with open(vocab_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    vocab = data.get('vocab', {})
    
    # Get the maximum index value in the vocab
    if vocab:
        max_index = max(vocab.values())
    else:
        max_index = -1  # Start from 0 if vocab is empty
    
    # For each new phoneme, add it to the vocab if not present
    for phoneme in new_phonemes:
        if phoneme not in vocab.keys():
            max_index += 1
            vocab[phoneme] = max_index
    
    # Update the data
    data['vocab'] = vocab
    
    # Write back to the file
    with open(vocab_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)


def checkPhonemeList(IPAlist, vocab_file_path="./vocab.json"):
    not_in_vocab = set()

    with open(vocab_file_path, 'r', encoding='utf-8') as f:
        #print(f.read())
        f.seek(0)  # Reset file pointer to beginning
        vocab_data = json.load(f)

    vocabIPAs = list(vocab_data['vocab'].keys())
    for IPAchar in IPAlist:
        if IPAchar not in vocabIPAs:
            not_in_vocab.add(IPAchar)

    f.close()
    
    if len(not_in_vocab) > 0:
        print(f"The following characters are not in the vocabulary: {', '.join(not_in_vocab)}")
        update_vocab(not_in_vocab)


def phoneme2token(phonemes, vocab_file_path="./vocab.json"):
        tokens = []

        with open(vocab_file_path, 'r') as f:
            #print(f.read())
            f.seek(0)  # Reset file pointer to beginning
            vocab_data = json.load(f)

        vocab = vocab_data["vocab"]
    
        if isinstance(phonemes, list):
            for phone in phonemes:
                phone = phone.split("\t")[0]
                phonemes_split = phone.split("|")
                tokens.append(
                    [vocab[p] for p in phonemes_split if p in vocab.keys()]
                )
        else:
            phonemes = phonemes.split("\t")[0]
            phonemes_split = phonemes.split("|")
            tokens = [vocab[p] for p in phonemes_split if p in vocab.keys()]

        f.close()
        return tokens



@torch.no_grad()
def extract_semantic_code(semantic_mean, semantic_std, speech_16k):
    # Extract features
    inputs = processor(speech_16k, sampling_rate=16000, return_tensors="pt")
    input_features = inputs["input_features"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Get embeddings
    vq_emb = semantic_model(
        input_features=input_features,
        attention_mask=attention_mask,
        output_hidden_states=True,
    )
    feat = vq_emb.hidden_states[17]  # (B, T, C)
    feat = (feat - semantic_mean) / semantic_std

    # Quantize to get semantic codes
    semantic_code, _ = semantic_codec.quantize(feat)  # (B, T)
    semantic_code = semantic_code.squeeze(0)  # Remove batch dimension

    return semantic_code.cpu().numpy()  # Convert to numpy array for saving
    

# Main dataset preparation function
def prepare_dataset(input_text_file, output_csv_file, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # Open the CSV file for writing
    with open(output_csv_file, mode='w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write header
        csv_writer.writerow(['phoneme_ids', 'semantic_code_path'])

        # Read the input text file
        with open(input_text_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()

            for line in tqdm(lines):
                line = line.strip()
                if not line:
                    continue  # Skip empty lines
                try:
                    audio_path, transcript = line.split('|')
                except ValueError:
                    print(f"Line format incorrect: {line}")
                    continue

                if "]¶[" in line:
                    continue

                # Load audio
                speech_16k, _ = librosa.load(audio_path, sr=16000)

                # Detect language
                #language = detect_text_language(transcript)
                #if language not in ["en", "zh", "ja", "ko", "fr", "de", "vi"]:
                #    print(f"Unsupported language '{language}' for transcript: {transcript}")
                #    continue

                language = "en"
                if LangLib.isCN(transcript):
                    language = "zh"

                if LangLib.isJaptxt(transcript):
                    language = "ja"

                if LangLib.isViet(transcript):
                    language = "vi"
                                
                # Convert transcript to phoneme tokens
                phonemes = g2p_(transcript, language)
                phonemeStr = phonemes[0]

                checkPhonemeList(phonemeStr.split('|'))

                phoneme_ids = phoneme2token(phonemeStr) # Phoneme IDs
                # Extract semantic codes
                semantic_code = extract_semantic_code(semantic_mean, semantic_std, speech_16k)
                # Save semantic code as .npy file
                semantic_code_filename = os.path.basename(audio_path).replace('.wav', '.npy').replace('.mp3', '.npy')
                semantic_code_path = os.path.join(output_dir, semantic_code_filename)
                np.save(semantic_code_path, semantic_code)
                # Write to CSV
                csv_writer.writerow([phoneme_ids, semantic_code_path])

    print("Dataset preparation completed.")

# Usage
#input_text_file = DatasetDir + 'dataset.txt'  # Replace with your input text file path
output_csv_file = DatasetDir + 'prepared_data.csv'  # Output CSV file
output_dir = DatasetDir + 'semantic_codes'  # Directory to save semantic code .npy files

prepare_dataset(input_text_file, output_csv_file, output_dir)


In [ ]:

import os
# Anyone else should probably change this to 0 if they only have one GPU. 
# We're only putting it on our secondary GPU since it has alot of VRAM. Training this thing takes ALOT of VRAM.
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import ast  # For safely evaluating strings of lists
import safetensors
from utils.util import load_config
from pytorch_optimizer import AdamW

from tqdm import tqdm
import torch.nn.functional as F

import torch.optim as optim
from maskgct_t2s import MaskGCT_T2S
from safetensors.torch import load_file

###########  ------------------

padding_token = 0  # Adjust if a different padding token is used
model_type = "t2s_model"
ckptDir = f"./ckpt/MaskGCT/{model_type}/"

# Training loop
num_epochs = 700
batchSize = 1  #looks like it might only be trained like this...
warmup_steps = 100  # is it a good idea to copy F5-TTS on this? who knows!
# -------------------

# Initialize necessary models and processors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Load semantic codec
cfg_path = "./maskgct/config/maskgct.json"
cfg = load_config(cfg_path)



class T2SDataset(Dataset):
    def __init__(self, csv_file, max_semantic_length=11600):
        """
        Args:
            csv_file (str): Path to the CSV file with phoneme IDs and semantic code paths.
            max_semantic_length (int): Maximum allowed length for the semantic code sequence.
        """
        import pandas as pd
        self.data = pd.read_csv(csv_file)
        self.max_semantic_length = max_semantic_length

        # Pre-load data and filter out samples that exceed the maximum length
        self.filtered_data = []
        for _, row in self.data.iterrows():
            phoneme_ids = ast.literal_eval(row['phoneme_ids'])
            semantic_code_path = row['semantic_code_path']

            # Load semantic code
            if os.path.exists(semantic_code_path):
                semantic_code = np.load(semantic_code_path)
                if len(semantic_code) <= self.max_semantic_length:
                    # Append valid sample to filtered data
                    self.filtered_data.append(
                        {
                            'phoneme_ids': phoneme_ids,
                            'semantic_code': semantic_code
                        }
                    )

    def __len__(self):
        return len(self.filtered_data)

    def __getitem__(self, idx):
        # Retrieve the pre-loaded data
        sample = self.filtered_data[idx]
        phoneme_ids = torch.tensor(sample['phoneme_ids'], dtype=torch.long)
        semantic_code = torch.tensor(sample['semantic_code'], dtype=torch.long)

        return {
            'phoneme_ids': phoneme_ids,
            'semantic_code': semantic_code
        }



def collate_fn(batch):
    #batch = [item for item in batch if item is not None]
    # Extract phoneme_ids and semantic_code from each item
    phoneme_ids = [item['phoneme_ids'] for item in batch]
    semantic_codes = [item['semantic_code'] for item in batch]

    # Pad sequences
    phoneme_ids_padded = torch.nn.utils.rnn.pad_sequence(
        phoneme_ids, batch_first=True, padding_value=1023  # Padding index for phoneme IDs
    )
    semantic_codes_padded = torch.nn.utils.rnn.pad_sequence(
        semantic_codes, batch_first=True, padding_value=-1  # Padding index for semantic codes
    )

    # Create masks
    phone_mask = (phoneme_ids_padded != 1023).long()
    semantic_mask = (semantic_codes_padded != -1).long()

    return {
        'phone_ids': phoneme_ids_padded,
        'phone_mask': phone_mask,
        'semantic_code': semantic_codes_padded,
        'semantic_mask': semantic_mask
    }


# Usage
csv_file = './dataset/prepared_data.csv'  # Path to your CSV file


# Training loop

# Initialize model
cfg = load_config(cfg_path).model.t2s_model
model = MaskGCT_T2S(cfg=cfg).to(device)

CodeBookSize = cfg.cond_codebook_size

print(cfg)
# Define optimizer and loss function

optimizer = AdamW(
    model.parameters(),
    lr=1e-5,
)# Yeah just put the optimizer you pick here

criterion = torch.nn.CrossEntropyLoss()  # Adjust ignore_index as needed


# Define the loss function with reduction='none' since we'll compute loss manually
#criterion = torch.nn.KLDivLoss(reduction='none')



checkpoints = [f for f in os.listdir(ckptDir) if f.endswith('.pt')]
if checkpoints:
    ckpt_path = sorted(checkpoints, key=lambda x: os.path.getmtime( os.path.join(ckptDir,x )))[-1]
else:
    ckpt_path = "origmodel.safetensors"
ckpt_path = os.path.join(ckptDir, ckpt_path)
print(ckpt_path)


ckpt_type = ckpt_path.split(".")[-1]
if ckpt_type == "safetensors":
    checkpoint = load_file(ckpt_path, device="cuda")
    model.load_state_dict(checkpoint)
else:
    checkpoint = torch.load(ckpt_path, map_location=device, weights_only=True)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['opt'])    
# Load pre-trained weights into the model
#safetensors.torch.load_model(model, ckpt_path)


# Create dataset and dataloader
dataset = T2SDataset(csv_file)
dataloader = DataLoader(dataset, batch_size=batchSize, shuffle=True, collate_fn=collate_fn)

step = 0


total_steps = len(dataloader) * num_epochs
decay_steps = total_steps - warmup_steps

warmup_scheduler = LinearLR(optimizer, start_factor=1e-8, end_factor=1.0, total_iters=warmup_steps)
decay_scheduler = LinearLR(optimizer, start_factor=1.0, end_factor=1e-8, total_iters=decay_steps)

scheduler = SequentialLR(optimizer, schedulers=[warmup_scheduler, decay_scheduler], milestones=[warmup_steps])


progress_bar = tqdm(dataloader, unit="step")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in progress_bar:
        optimizer.zero_grad()

        # Move data to device
        x0 = batch['semantic_code'].to(device)
        x_mask = batch['semantic_mask'].to(device)
        phone_ids = batch['phone_ids'].to(device)
        phone_mask = batch['phone_mask'].to(device)

        # Ohh you need the x_mask and phone_mask for batches and ignoring padded tokens! RIGHT!
        
        # Forward pass
        logits, final_mask, _, prompt_len, mask_prob = model(
            x0, x_mask, phone_ids, phone_mask
        )

        ## Prepare the final mask
        #final_mask = final_mask.squeeze(-1)  # Shape: (B, T)
        ## Flatten the final mask
        #final_mask_flat = final_mask.view(-1)  # Shape: (B*T,)
        ## Flatten targets
        #targets_flat = x0_targets.view(-1)  # Shape: (B*T,)
        ## Compute model's predicted distribution p_probs
        #logits_flat = logits.view(-1, logits.size(-1))  # Shape: (B*T, vocab_size)
        #p_probs_flat = F.log_softmax(logits_flat, dim=-1)  # Log probabilities for KLDivLoss
        ## Compute true distribution q_probs as one-hot vectors over x0_targets
        #vocab_size = logits.size(-1)
        #q_probs_flat = torch.zeros_like(p_probs_flat)  # Shape: (B*T, vocab_size)
        #q_probs_flat.scatter_(1, targets_flat.unsqueeze(1), 1.0)  # One-hot encoding
        ## Compute KL divergence only over masked positions
        ## Masked positions have final_mask_flat == 1
        #kl_div = criterion(p_probs_flat, q_probs_flat).sum(dim=1)  # Shape: (B*T,)
        ## Apply the final mask
        #kl_div = kl_div * final_mask_flat  # Zero out unmasked positions
        ## Compute average loss
        #loss = kl_div.sum() / final_mask_flat.sum()

        #use reshape to flatten tensors
        #print(x0.shape)
        logits_flat = logits.reshape(-1, CodeBookSize)
        #print(logits_flat.shape)
        x0_flat = x0.reshape(-1)
        final_mask = final_mask.reshape(-1)
        #print(final_mask)
        #mask out the losses
        logits_flat = logits_flat[final_mask > 0]
        x0_flat = x0_flat[final_mask > 0]
        #print(logits_flat)
        # Compute the loss only over masked positions
        loss = criterion(logits_flat, x0_flat)
        #loss = torch.nn.functional.cross_entropy(logits_flat, x0_flat, reduction='none')
        #loss = loss.view(x0_targets.size()) 

        loss.backward()
        optimizer.step()
        scheduler.step()

        step += 1
        
        total_loss += loss.detach().item()
        progress_bar.set_postfix(step=str(step), loss=loss.detach().item())
        #print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.detach().item()}")

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")

    # Optionally, save the model checkpoint
    #torch.save(model.state_dict(), f'maskgct_t2s_epoch{epoch+1}.pth')

    # Save model and optimizer state dictionaries
    if (epoch + 1) % 10 == 0:  # Save every 10 epochs
        torch.save({
            #'epoch': epoch,
            'model': model.state_dict(),
            'opt': optimizer.state_dict(),
            #'loss': loss,
        }, ckptDir + 'newT2S_epoch{}.pt'.format(epoch))



In [ ]:
# The below is just debug and experimental.... pretty much this whole thing is debug and experimental

# This is not meant to be user friendly. No time, no resources. Busy. 

In [ ]:
print(p_probs_flat.shape)
print(q_probs_flat.shape)
print(criterion(p_probs_flat, q_probs_flat).shape)

In [ ]:
def get_dataset_statistics(dataset):
    phoneme_lengths = []
    semantic_code_lengths = []

    for i in range(len(dataset)):
        sample = dataset[i]
        phoneme_lengths.append(len(sample['phoneme_ids']))
        semantic_code_lengths.append(len(sample['semantic_code']))

    # Convert lists to numpy arrays for easier statistical calculations
    phoneme_lengths = np.array(phoneme_lengths)
    semantic_code_lengths = np.array(semantic_code_lengths)

    # Calculate statistics
    stats = {
        'phoneme_lengths': {
            'max': phoneme_lengths.max(),
            'min': phoneme_lengths.min(),
            'mean': phoneme_lengths.mean(),
            'median': np.median(phoneme_lengths),
            'std': phoneme_lengths.std(),
        },
        'semantic_code_lengths': {
            'max': semantic_code_lengths.max(),
            'min': semantic_code_lengths.min(),
            'mean': semantic_code_lengths.mean(),
            'median': np.median(semantic_code_lengths),
            'std': semantic_code_lengths.std(),
        }
    }

    # Find the longest entry
    longest_phoneme_idx = phoneme_lengths.argmax()
    longest_phoneme_sample = dataset[longest_phoneme_idx]
    longest_semantic_idx = semantic_code_lengths.argmax()
    longest_semantic_sample = dataset[longest_semantic_idx]

    return stats, longest_phoneme_sample, longest_semantic_sample

# Get statistics and longest entries
stats, longest_phoneme_sample, longest_semantic_sample = get_dataset_statistics(dataset)

print("Statistics:")
print(stats)

print("\nLongest phoneme sequence:")
print("Length:", len(longest_phoneme_sample['phoneme_ids']))
print("Phoneme IDs:", longest_phoneme_sample['phoneme_ids'])

print("\nLongest semantic code sequence:")
print("Length:", len(longest_semantic_sample['semantic_code']))
print("Semantic Codes:", longest_semantic_sample['semantic_code'])


In [ ]:
import os
import torch
from safetensors.torch import save_file

model_type = "t2s_model"
ckptDir = f"./ckpt/MaskGCT/{model_type}/"

checkpoints = [f for f in os.listdir(ckptDir) if f.endswith('.pt')]
if checkpoints:
    ckpt_path = sorted(checkpoints, key=lambda x: os.path.getmtime( os.path.join(ckptDir,x )))[-1]
else:
    ckpt_path = "origmodel.safetensors"
ckpt_path = os.path.join(ckptDir, ckpt_path)
print(ckpt_path)
checkpoint = torch.load(ckpt_path, map_location='cpu')
# Assuming the checkpoint contains a 'state_dict'
state_dict = checkpoint['model'] if 'model' in checkpoint else checkpoint
# Save the state dictionary to .safetensors format
save_file(state_dict, ckptDir + 'model.safetensors')

In [ ]:
save_file(state_dict['model'], ckptDir + 'model.safetensors')